# Initial Readings

Initial references: 

> Pope, B. J., Mahmood, K., Jung, C.-H., Georgeson, P. & Park, D. J. Single nucleotide-level mapping of DNA double-strand breaks in human HEK293T cells. Genomics Data doi:10.1016/j.gdata.2016.11.007 (2016).

> Tchurikov et al. (2016) Scientific Reports 6:25866 (doi:10.1038/srep25866);

> Tchurikov et al. (2015) Journal of Molecular Cell Biology 7(4):366-382 (doi:10.1093/jmcb/mju038).


Looking at the TCGA and ICGC databases, there are certain protocols that need to be done before data can be downloaded. 

For ICGC, this seems to be a good reference: http://docs.icgc.org/cloud/guide/#authorization. 

For GDC, this seems to be a good reference: https://docs.gdc.cancer.gov/Data_Transfer_Tool/Users_Guide/Data_Download_and_Upload/

Broad area: Correlations between gamma-H2AX or H2AX binding sites and breast cancer mutations in TCGA or ICGC datasets.

## Annotated Bibliography

> Review of gamma-H2AX and DSBs http://iv.iiarjournals.org/content/22/3/305.full.pdf

From here, it appears the H2AX phosphorylation occurs within a fairly large radius of the DSB (a few megabases?), with the apparent role to attract proteins for repair of the DSB. Upon repair, H2AX is dephosphorylated. There is discussion of methods of identifying gamma-H2AX through FISH or other methods, but that doesn't seem as relevant to this...

Kuo and Yang discuss the relationship of H2AX with cancer cells; where there is a knockout of the responsible gene, tumours rapidly developed (which appears consistent with the role of H2AX to attract proteins for DNA repair). It appears the the H2AX gene is frequently mutated in some cancers. 

There is extensive discussion of H2AX with regard to radiation that you should revisit.

> gamma-H2AX and cancer in general https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3094856/

More extensive discussion into the background behind H2AX. H2AX is a particular member of the H2A family of histones (there are five families; four "core" families, from each of which two members are selected and incorporated into an 8-histone core for a nucleosome, and a fifth family which acts in a linking role between nucleosomes. H2AX constitutes a varying proportion of the H2A core histones (e.g. 2% in lymphocytes and HeLa, 10% in fibroblasts, 20% in a particular glioma cell line...). There is greater discussion into the translation of H2AX from the H2AFX gene which may be useful.

The phosphorylation of H2AX occurs on an order of megabases from the DSB. 

> H2AX roles https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4357700/



### Other Fundamental Concepts

ChIP: appears to be a method of determining the locations of histone modifications (or other chromatin-binding proteins) in the genome. Based on precipitating only chromatin that binds to the protein, then sequencing this to determine the location within the genome. 

CIGAR: a string produced after alignment of reads to a reference sequence which describes how the read "fits" against the reference.

# Plan

The basic plan as I understand today (2017/06/26) is to:

1. Find .bed data (or other data that can be converted to .bed) recording the coordinates/binned frequency of H2AX and gamma-H2AX in various primary tissue types. You'll neeed to search for H2AX data tracks, and confrim what tissue they are...A previous 2014 paper - GSE44309 accession https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE44309m ...check?
2. (Extract from TCGA/GDC data of coordinates of somatic mutations involved in breast cancer. Using the clinical/histopathology data, the somatic mutations can be separated based on features e.g. estrogen receptor positive vs negative, that can be used to explore the different relationship based on subtype). Khalid has already done this.
3. Filter out, based on annotations, repeat mask regions or overlaps or complex mappings (get this from Encode?) (this step I am not entirely clear on)
4. Using bedtools, find the intersection between the coordinates of H2AX and the somatic mutation sites (or some intersection within a forgivable range)
5. Compare the intersection/correlation with that of a control, which consists of either 1) randomly sampled locations in the genome (after filtering), or some other tissue or 2) based on Tchurikov's group for marks not correlating with raft sites (this part I am also not entirely clear on...). Maybe if I understand correctly - we should choose RANDOM coordinates which are NOT H2AX binding sites and then see if these match with the somatic mutations. 

Basically, you should have two main sets of .bed files - one set of somatic mutations (based on varying stratification...) and one set of H2AX/gamma-H2AX (based on varying tissue types). Then you somehow need to do something regarding the control...

In addition - when doing this, you MUST make sure that the .bed files you use are coordinates with respect to the same genome reference (**hg19** is the one to look for). 

Some fundamental concepts which I need to learn/revise are:

- awk [looked at]
- sed [looked at]
- Linux commands and data pipelines (as well as how to get this working on a windows computer...) [looked at]
- bedtools or pybedtools [Successfully installed bedtools with CygWin! No luck with pybedtools...]
- repeat mask/other data processing of bed files...

CONCRETE TASKS

1. Download GSE25577 data and see how it looks...(Optionally download GSE44309 data...)
2. Ask Khalid regarding using the estrogen receptor positive mutations vs estrogen receptor negative mutations (or how he extracted this as it looked controlled)
3. Read up on the filtering process required and ask Khalid if necessary
4. Use bedtools to find the intersection between the coordinates of H2AX and consider some way of visualising this
5. Consider a control and ask Khalid about this if still unclear

## 1: Sourcing H2AX Data

I have (not yet downloaded since the file, 1.2GB, wouldn't download completely on my connection yet) the raw .bed data from GSE25577 which appears to contain H2AX and $\gamma$-H2AX coordinates for CD4+ and Jurkat cells (among another one). According to the paper (https://academic.oup.com/nar/article-lookup/doi/10.1093/nar/gks287), "sequence reads were mapped to the human genome [University of California, Santa Cruz (UCSC) **hg18** assembly based on National Center for Biotechnology Information (NCBI) build 36.1] by means of the Illumina sequencing pipeline." 

> What is the meaning of "strand" in the .bed file? If it's a histone, how does this occur at a strand?

> GSE44309 also looks like it may be of use, but I cannot download it at the moment due to its large size. Note that this one is mapped to the **hg19** assembly.

> Can this be used for breast cancer mutations? I can see how it might be okay since the genome is the same regardless of the cell...but the only thing I do not know is if the phosphorylation of H2AX is cell type dependent or...well I'm really not sure at all to be honest.

Other ones to look at:...(I don't know about this very well...)
GSE97589
Maybe search from here later... https://www.ncbi.nlm.nih.gov/gds

I extracted the .bed files and ran them through liftOver (https://genome.ucsc.edu/cgi-bin/hgLiftOver) to convert from human hg18 to human hg38 (as the GDC data is) with the default settings. This was saved as preprocessed data. 


## 2: Sourcing Mutation Data

ICGC's FAQ states that is uses **hg19**; GDC states that it uses **hg38**. Neither of these match the H2AX data available...

> Is it possible to remap the hg18 data to the hg19/hg38 data? e.g. is the NCBI Genome Remapping Tool sufficient to remap?.. "Liftover..."

I queried GDC for data files:

```
Primary Site IS Breast AND Data Type IS Masked Somatic Mutation
```

which returned four results, the difference between which look to be the "Workflow Type" (MuSE, MuTect, SomaticSniper, VarScan) - I'm not quite sure beyond this though. I think I will stick with one for the moment, using VarScan as an arbitrary initial attempt - and hopefully the code I write is generalisable enough that I can do the others without too much hassle.

[I think I will need to download the Clinical Supplement and Biospecimen Supplement later (maybe?), but at this stage, I'll leave it for now.]

Okay, I've done this, and downloaded the Clinical Supplement and Biospecimen Supplement with 
```
Primary Site IS Breast AND Data Type IS [Clinical Supplement/Biospecimen Supplement]
```

## 3: Filtering Data

Pope et al.'s *Genomics Data* paper uses multiple filters:
- The ENCODE blacklist for the reference genome
- Low-complexity regions derived from RepeatMasker
- (Unknown filtering steps in raft_bed_2sites_parse.py)
- "Reads that mapped in either orientation were treated as likely to be erroneous if the CIGAR string showed evidence of clipping at either terminus." 
- Mapping qualities needed to be > 40
- Sites within 5 base pairs of the Sau consensus site  were removed)

# DRAFT SCRIPTS

I think these will be much easier to write and debug when I have access to a linux computer. But I will attempt at writing these scripts now so I have something to base off...



### Remap hg18 to h38 

This will need to be done using liftOver...I'm not entirely confident to be perfectly honest but I'll need to try my best. 

### Convert Mutation Annotation Format (from GDC) to BED

This is extremely simple, literally a matter of extracting columns...

```{bash}

cut -f 5-8 [maf file] | awk 'NR >= 6 {print}' | awk '$3 = $3 FS "." FS "."' >> [new bed file]

```

(Cut the 5th to 8th (includes both ends) fields of lines and take only rows numbered 6 or greater (rows 1 to 4 are metadata and row 5 is the column header), then replace column 3 with column 3 + . + . (i.e. two new columns populated with "."). 

> I think this produces a space-delimited bed file, so I need to fix this...

### Filter Out Blacklist

The hg38 blacklist was available from here: http://mitra.stanford.edu/kundaje/akundaje/release/blacklists/hg38-human/. 

> RepeatMasker - need to check how to do this.

Once this is done, you will need to have single bed file representing your blacklist.

then you should simply be able to do:

```{bash}
bedtools subtract -a [input .bed file] -b [blacklist .bed file] >> [new bed file] 
```

### Window

```{bash}
bedtools window -a [marker .bed] -b [mutations .bed] -w 1000 >> [new bed file]
```

### Counting

From here, I guess you could count the proportion of matches... 